In [1]:
print('ok')

ok


In [18]:
import chromadb
import google.generativeai as genai
import pandas as pd
import os
import time  # <-- IMPORT THE TIME CULTIVATION METHOD
from dotenv import load_dotenv
from chromadb import Documents, EmbeddingFunction, Embeddings

/home/aditya/anaconda3/envs/venvGenAI1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
COLLECTION_NAME = "legal_docs"
GEMINI_EMBEDDING_MODEL = "models/text-embedding-004"

we again need to use gemini embedding function just with retrieval task

In [20]:
# Load environment variables from a .env file
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Please set it in your .env file.")

# Configure the Gemini client
genai.configure(api_key=GOOGLE_API_KEY)

In [21]:
# --- 2. Define the Custom Gemini Embedding Function ---
class GeminiEmbeddingFunction(EmbeddingFunction):
    """
    Custom embedding function using the Gemini API, with an __init__ method.
    """
    def __init__(self):
        pass

    def __call__(self, input: Documents) -> Embeddings:
        result = genai.embed_content(model=GEMINI_EMBEDDING_MODEL,
                                         content=input,
                                         task_type="RETRIEVAL_QUERY")
        return result['embedding']
        

In [22]:
gemini_ef = GeminiEmbeddingFunction()

In [2]:
import chromadb

In [23]:
client = chromadb.PersistentClient(path="/home/aditya/0Legal_Agent_Project/chroma_db")

In [24]:
client.list_collections()

[Collection(name=legal_docs)]

In [25]:
collection = client.get_collection(name="legal_docs", embedding_function=gemini_ef)

In [26]:
collection.get('nyaaya_10_chunk_0')

{'ids': ['nyaaya_10_chunk_0'],
 'embeddings': None,
 'documents': ['Home/Legal Explainers/Money & Property/Money/ChequesA cheque allows a person to ask their bank to deduct money from their account to give it to someone. There are many types of cheques such as order cheque, crossed cheque, bearer cheque etc.ChequesThis explainer discusses the concept of cheques, cheque bouncing and punishments for committing cheque related crimes. It primarily deals with the law laid out in the Negotiable Instruments Act, 1881.Introduction to ChequesHow do cheques work?Intention of Drawer of ChequeImportance of Signature on a ChequeEncash a chequeCheque ClearingCheque Truncation SystemEndorsing chequesSpeed Clearing for Outstation ChequesTypes of ChequesValid chequesCrossed ChequeUncrossed or open chequeBearer chequeOrder ChequeCancelled chequePrecautionsPrecautions to be taken by customersPrecautions to be taken by banks while dealing with chequesProblems & ForgeryCheque Processing ProblemsForging a c

In [29]:
res = collection.query(
    query_texts=["What is are the types of cheques?"],
    n_results=5
)

In [30]:
res

{'ids': [['kaggle_1780_chunk_0',
   'nyaaya_10_chunk_0',
   'kaggle_1904_chunk_0',
   'kaggle_1780_chunk_3',
   'kaggle_1902_chunk_0']],
 'embeddings': None,
 'documents': [['A “cheque” is a bill of exchange drawn on a specified banker and not expressed to be payable otherwise than on demand and it includes the electronic image of a truncated cheque and a cheque in the electronic form.',
   'Home/Legal Explainers/Money & Property/Money/ChequesA cheque allows a person to ask their bank to deduct money from their account to give it to someone. There are many types of cheques such as order cheque, crossed cheque, bearer cheque etc.ChequesThis explainer discusses the concept of cheques, cheque bouncing and punishments for committing cheque related crimes. It primarily deals with the law laid out in the Negotiable Instruments Act, 1881.Introduction to ChequesHow do cheques work?Intention of Drawer of ChequeImportance of Signature on a ChequeEncash a chequeCheque ClearingCheque Truncation Sy